In [23]:
import pandas as pd

In [25]:
df = pd.read_csv(r"C:\Users\Admin\Desktop\Project-1\Climate_dataset.csv")

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import StandardScaler
from sklearn.linear_model   import LinearRegression
from sklearn.ensemble       import RandomForestRegressor
import xgboost               as xgb
from sklearn.metrics        import mean_squared_error, r2_score

# 1) Define feature columns (drop Year entirely)
feature_cols = [c for c in df.columns if c not in ('Year','Temperature_C')]

# 2) Split X/y
X = df[feature_cols]
y = df['Temperature_C']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# 3) Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

# 4) Train your models again
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest'    : RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost'          : xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
}

results = {}
for name, mdl in models.items():
    mdl.fit(X_train_scaled, y_train)
    pred_val = mdl.predict(X_val_scaled)
    mse = mean_squared_error(y_val, pred_val)
    results[name] = {'R2': r2_score(y_val, pred_val), 'RMSE': np.sqrt(mse)}

results_df = pd.DataFrame(results).T
print(results_df)

# Pick the best
best_name  = results_df['R2'].idxmax()
best_model = models[best_name]
print("Best model:", best_name)


# 5) Scenario Analysis (no Year)
horizon = 10
years   = np.arange(df['Year'].max()+1, df['Year'].max()+1+horizon)

# last known physical drivers
last = df[feature_cols].iloc[-1]

# define CO₂ scenarios
scenarios = {
    'Flat CO₂'    : np.zeros(horizon),
    '+2 ppm/year' : np.full(horizon, 2),
    '+5 ppm/year' : np.full(horizon, 5),
}

projections = {}
for name, delta in scenarios.items():
    co2_vals = last['CO2_ppm'] + np.cumsum(delta)
    df_s     = pd.DataFrame({col: last[col] for col in feature_cols}, index=years)
    df_s['CO2_ppm'] = co2_vals
    X_s            = scaler.transform(df_s)           # now exactly 7 features
    projections[name] = best_model.predict(X_s)

# 6) Plot
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.lineplot(x=df['Year'], y=df['Temperature_C'], label='Historical', color='black')
for name, pred in projections.items():
    sns.lineplot(x=years, y=pred, label=name, linestyle='--')

plt.title("Temperature Projections Under Different CO₂ Scenarios")
plt.xlabel("Year")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.grid(True)
plt.show()


                          R2      RMSE
Linear Regression -32.985238  0.263950
Random Forest     -58.112579  0.348110
XGBoost           -76.362004  0.398236
Best model: Linear Regression


In [28]:
import joblib

# Save model and scaler to disk
joblib.dump(best_model, 'best_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [29]:
import streamlit as st
import numpy as np
import pandas as pd
import joblib

# Load model and scaler
model = joblib.load('best_model.pkl')
scaler = joblib.load('scaler.pkl')

# Streamlit UI
st.title("🌍 Climate Temperature Prediction App")

st.markdown("""
This app predicts the **average temperature (°C)** based on climate-related variables such as CO₂ levels, sea levels, and extreme weather events.
""")

# Define features used in the model (must match training!)
feature_cols = ['CO2_ppm', 'Sea_Level_mm', 'Extreme_Weather_Events', 'Temp_Anomaly_C', 'Sea_Level_Rise_mm']

# Create input fields
inputs = {}
for col in feature_cols:
    inputs[col] = st.number_input(f"Enter value for {col}", step=0.1)

# Predict on button click
if st.button("Predict Temperature"):
    input_df = pd.DataFrame([inputs])
    scaled_input = scaler.transform(input_df)
    prediction = model.predict(scaled_input)[0]
    st.success(f"🌡️ Predicted Average Temperature: {prediction:.2f} °C")


In [30]:
import joblib

joblib.dump(best_model, 'best_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("✅ Saved best_model.pkl and scaler.pkl")


✅ Saved best_model.pkl and scaler.pkl


In [35]:
import os
print(os.getcwd())
print(os.listdir())


C:\Users\Admin\Desktop\Project-1
['.ipynb_checkpoints', 'best_model.pkl', 'Climate Change Modeling.pdf', 'Climate_dataset.csv', 'climate_nasa.csv', 'GLB.Ts+dSST.csv', 'Project-1.ipynb', 'requirements.txt', 'scaler.pkl', 'Untitled.ipynb']
